<a href="https://colab.research.google.com/github/ssandanshi/Speaker-Recognition/blob/master/Speaker_Recognition_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=1c77aabf0f8896b3334023f117734a0091dbefe9f1ae347f27f15f5b1fc90e3c
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
import os
import glob
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import keras.backend as K
import keras
from python_speech_features import fbank, delta
import operator
from collections import OrderedDict
from multiprocessing import Pool
from time import time
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from shutil import copyfile
import pickle

Using TensorFlow backend.


In [0]:
sample_rate = 12000

In [0]:
def create_catalogue(path,pattern="*/*.wav",flag=False):
  data = pd.DataFrame()
  data['filename'] = glob.glob(os.path.join(path, pattern), recursive=True)
  if not flag:
    data['class'] = data['filename'].apply(lambda x: x.split("/")[1])
  else:
    data['class'] = data['filename'].apply(lambda x: x.split("/")[1].split('_')[0])
  num_speakers = len(data['class'].unique())
  print('Found {} files with {} different speakers.'.format(str(len(data)), str(num_speakers)))
  # print(data.head(10))
  return data

In [0]:
NUM_FRAMES=399
BATCH_SIZE=32
TRIPLET_PER_BATCH=3

# Test on test set

In [0]:
!cp drive/My\ Drive/vox_test_npy.zip .
!unzip -q vox_test_npy.zip

In [0]:
!ls

drive  sample_data  vox_test_npy  vox_test_npy.zip


In [0]:
test_catalog = create_catalogue("vox_test_npy","*.npy",flag=True)
test_catalog

Found 4874 files with 40 different speakers.


,filename,class
0,vox_test_npy/id10292_29.npy,id10292
1,vox_test_npy/id10297_53.npy,id10297
2,vox_test_npy/id10276_76.npy,id10276
3,vox_test_npy/id10292_75.npy,id10292
4,vox_test_npy/id10283_14.npy,id10283
...,...,...
4869,vox_test_npy/id10300_258.npy,id10300
4870,vox_test_npy/id10307_148.npy,id10307
4871,vox_test_npy/id10277_60.npy,id10277
4872,vox_test_npy/id10276_23.npy,id10276


In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le2 = LabelEncoder()
test_catalog["class"] = le2.fit_transform(test_catalog["class"])

In [0]:
le2.classes_

array(['id10270', 'id10271', 'id10272', 'id10273', 'id10274', 'id10275',
       'id10276', 'id10277', 'id10278', 'id10279', 'id10280', 'id10281',
       'id10282', 'id10283', 'id10284', 'id10285', 'id10286', 'id10287',
       'id10288', 'id10289', 'id10290', 'id10291', 'id10292', 'id10293',
       'id10294', 'id10295', 'id10296', 'id10297', 'id10298', 'id10299',
       'id10300', 'id10301', 'id10302', 'id10303', 'id10304', 'id10305',
       'id10306', 'id10307', 'id10308', 'id10309'], dtype=object)

In [0]:
def predict_model_batch(model,inp,batch_size=10):
  i = 0
  embedding = None
  while i < len(inp) and i+batch_size<len(inp):
    subset = inp[i:i+batch_size]
    sub_embed = model.predict_on_batch(subset)
    if embedding is None:
        embedding = sub_embed
    else:
        embedding = np.concatenate([embedding, sub_embed], axis=0)
    i+=batch_size
  if i<len(inp):
    subset = inp[i:]
    sub_embed = model.predict_on_batch(subset)
    if embedding is None:
        embedding = sub_embed
    else:
        embedding = np.concatenate([embedding, sub_embed], axis=0)
  return embedding

In [0]:
def split_balanced_and_load_data(data,samples_per_class,model=None,batch_size=10,top_classes=None,seed=999):
  if top_classes is None:
    classes = data["class"].unique()
  else:
    classes = data["class"].value_counts()[:top_classes].index.to_list()
    data = data[data["class"].isin(classes)]
  train = pd.DataFrame()
  removed_indices = []
  for cl in classes:
    data_class = data[data["class"]==cl]
    if samples_per_class<len(data_class)/2:
      sample = data_class.sample(samples_per_class,random_state=seed)
      removed_indices.extend(sample.index.to_list())
      train = pd.concat([train,sample], axis=0)
    else:
      sample = data_class.sample(int(len(data_class)/2),random_state=seed)
      removed_indices.extend(sample.index.to_list())
      train = pd.concat([train,sample], axis=0)
        
  test = data.loc[~data.index.isin(removed_indices)]
  train = train.reset_index(drop=True)
  test = test.reset_index(drop=True)
  trainX, trainY, testX, testY = np.array(train["filename"]),np.array(train["class"]),np.array(test["filename"]),np.array(test["class"])

  if model is not None:
    trainX = predict_model_batch(model,np.array([np.load(x) for x in trainX]),batch_size=batch_size)
    testX = predict_model_batch(model,np.array([np.load(x) for x in testX]),batch_size=batch_size)
  else:
    trainX = np.array([np.load(x) for x in trainX])
    testX = np.array([np.load(x) for x in testX])
    trainX = np.reshape(trainX,(trainX.shape[0],trainX.shape[1]*trainX.shape[2]*trainX.shape[3]))
    testX = np.reshape(testX,(testX.shape[0],testX.shape[1]*testX.shape[2]*testX.shape[3]))
  return trainX, trainY, testX, testY




In [0]:
best_model = None
samples_per_class = 80
num_classes = len(test_catalog["class"].unique())
top_classes = None

In [0]:
trainX, trainY, testX, testY = split_balanced_and_load_data(test_catalog,samples_per_class,best_model,batch_size=20,top_classes=top_classes)

In [0]:
print(trainX.shape,trainY.shape,testX.shape,testY.shape)

(2168, 25536) (2168,) (2706, 25536) (2706,)


In [0]:
trainX.shape[1]

25536

In [0]:
from sklearn.svm import LinearSVC
clf = LinearSVC()

In [0]:
clf.fit(trainX,trainY)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
y_pred_Linear_SVC = clf.predict(testX)

In [0]:
accuracy_score(testY,y_pred_Linear_SVC)

0.34146341463414637

In [0]:
logisticRegressionclf = LogisticRegression()

In [0]:
logisticRegressionclf.fit(trainX,trainY)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
accuracy_score(testY,logisticRegressionclf.predict(testX))

0.369179600886918

In [0]:
from sklearn.neural_network import MLPClassifier
mlpclf = MLPClassifier()
mlpclf.fit(trainX,trainY)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
accuracy_score(testY,mlpclf.predict(testX))

0.32852919438285294

In [0]:
from sklearn.svm import SVC
rbfSVCclf = SVC()
rbfSVCclf.fit(trainX,trainY)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
accuracy_score(testY,rbfSVCclf.predict(testX))

0.3632668144863267

In [0]:
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate
from numpy import random

In [0]:
inp = Input(shape=(trainX.shape[1], 1))
img_1 = Convolution1D(16, kernel_size=9, activation=activations.relu, padding="valid")(inp)
img_1 = Convolution1D(16, kernel_size=9, activation=activations.relu, padding="valid")(img_1)
img_1 = MaxPool1D(pool_size=16)(img_1)
img_1 = Dropout(rate=0.5)(img_1)
img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
img_1 = MaxPool1D(pool_size=4)(img_1)
img_1 = Dropout(rate=0.5)(img_1)
img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
img_1 = MaxPool1D(pool_size=4)(img_1)
img_1 = Dropout(rate=0.5)(img_1)
img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
img_1 = GlobalMaxPool1D()(img_1)
img_1 = Dropout(rate=0.5)(img_1)

dense_1 = Dense(64, activation=activations.relu)(img_1)
dense_1 = Dense(1028, activation=activations.relu)(dense_1)
dense_1 = Dense(len(le2.classes_), activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 25536, 1)          0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 25528, 16)         160       
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 25520, 16)         2320      
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 1595, 16)          0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 1595, 16)          0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 1593, 32)          1568      
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 1591, 32)          3104

In [0]:
model.fit(np.reshape(trainX,(trainX.shape[0],trainX.shape[1],1)),trainY,epochs=100)

Epoch 1/100
2168/2168 [==============================] - 3s 2ms/step - loss: 3.6483 - acc: 0.0295
Epoch 2/100
2168/2168 [==============================] - 2s 778us/step - loss: 3.6267 - acc: 0.0295
Epoch 3/100
2168/2168 [==============================] - 2s 800us/step - loss: 3.6199 - acc: 0.0286
Epoch 4/100
2168/2168 [==============================] - 2s 781us/step - loss: 3.6129 - acc: 0.0355
Epoch 5/100
2168/2168 [==============================] - 2s 788us/step - loss: 3.6138 - acc: 0.0355
Epoch 6/100
2168/2168 [==============================] - 2s 795us/step - loss: 3.6069 - acc: 0.0401
Epoch 7/100
2168/2168 [==============================] - 2s 752us/step - loss: 3.5522 - acc: 0.0507
Epoch 8/100
2168/2168 [==============================] - 2s 770us/step - loss: 3.5288 - acc: 0.0540
Epoch 9/100
2168/2168 [==============================] - 2s 800us/step - loss: 3.4815 - acc: 0.0581
Epoch 10/100
2168/2168 [==============================] - 2s 785us/step - loss: 3.4285 - acc: 0.0641
E

In [0]:
accuracy_score(testY,model.predict(np.reshape(testX,(testX.shape[0],testX.shape[1],1))).argmax(axis=-1))

0.09682187730968218